In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm
import random, pickle, math, warnings
import itertools,  multiprocessing, json
#warnings.simplefilter('ignore')
print("CPU Count: ", multiprocessing.cpu_count())

p = '/kaggle/input/santa-2024/sample_submission.csv'
df = pd.read_csv(p)

CPU Count:  96


In [2]:
#https://www.kaggle.com/docs/tpu
#https://www.kaggle.com/code/ryanholbrook/getting-started-with-tpus

#!pip uninstall -y tensorflow && pip install tensorflow-cpu
#import tensorflow as tf

#print("Tensorflow version " + tf.__version__)
#AUTO = tf.data.experimental.AUTOTUNE
# Detect TPU, return appropriate distribution strategy
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#print('Running on TPU ', tpu.master())
#tf.config.experimental_connect_to_cluster(tpu)
#tf.tpu.experimental.initialize_tpu_system(tpu)
#tpu_strategy = tf.distribute.TPUStrategy(tpu)
#print("REPLICAS: ", tpu_strategy.num_replicas_in_sync)

In [3]:
import transformers, torch, os
from math import exp

DEVICE = torch.device('cpu')
MODEL_PATH = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"

class PerplexityCalculator:
    def __init__(self,):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_PATH)
        self.model = transformers.AutoModelForCausalLM.from_pretrained(MODEL_PATH, device_map="auto", torch_dtype=torch.float32,)
        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
        self.model.eval()

    #add batch and multiprocessing again for CPU/GPU:}
    def get_perplexity(self, text: str) -> float:
        with torch.no_grad():
            text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"
            model_inputs = self.tokenizer(text_with_special, return_tensors='pt', add_special_tokens=False,)
            #model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}
            logits = self.model(**model_inputs, use_cache=True)['logits']
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = model_inputs['input_ids'][..., 1:].contiguous()
            loss = self.loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1))
            sequence_loss = loss.sum() / len(loss)
            loss_list = sequence_loss.cpu().item()
        return exp(loss_list)

# instantiating the model in the strategy scope creates the model on the TPU
#with tpu_strategy.scope():
     # define your model normally
scorer = PerplexityCalculator()

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 8/8 [00:02<00:00,  2.74it/s]


In [4]:
t = """reindeer mistletoe elf gingerbread family advent scrooge chimney fireplace ornament
reindeer walk gingerbread bake the night and sleep scrooge drive chimney jump elf laugh mistletoe give family advent fireplace ornament
magi yuletide cheer grinch carol holiday holly jingle naughty nice nutcracker polar beard ornament stocking chimney sleigh workshop gifts decorations
sleigh of the magi yuletide cheer is unwrap gifts and eat cheer holiday decorations holly jingle relax carol sing chimney visit grinch naughty nice polar beard workshop nutcracker ornament stocking
from and as have in not it of that the to we with you bow angel believe candle candy card chocolate cookie doll dream eggnog fireplace fruitcake game greeting hohoho hope joy kaggle merry milk night paper peace peppermint poinsettia puzzle season snowglobe star toy wreath wish workshop wonder wrapping
from and and as and have the in is it of of not that the to we with you advent card the angel bake beard believe bow candy candle carol cheer cheer chocolate chimney cookie decorations doll dream drive eat eggnog family fireplace fireplace chimney fruitcake game give gifts gingerbread greeting grinch holiday holly hohoho hope jingle jump joy kaggle laugh magi merry milk mistletoe naughty nice night night elf nutcracker ornament ornament wrapping paper peace peppermint polar poinsettia puzzle reindeer relax scrooge season sing sleigh sleep snowglobe star stocking toy unwrap visit walk wish wonder workshop workshop wreath yuletide"""

df['text'] = t.split('\n')
df['score'] = df['text'].map(lambda x: scorer.get_perplexity(x))
df.to_csv("submission.csv", index=False)
print(np.mean(df['score']))
df['score']

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


262.95834068043825


0    468.499121
1    463.726354
2    303.031473
3    209.184454
4     98.260833
5     35.047809
Name: score, dtype: float64

In [5]:
past = {}

def part_perm_brutem(st, start=0, end=3, skips=1, tol=1.0, smax=300):
    global past
    bestt = st
    best = scorer.get_perplexity(st)
    st = st.split(' ')
    part = st[start:end]
    if start>0:
        st1 =  ' '.join(st[:start]) + ' '
    else:
        st1 = ''
    if end<len(st): 
        st2 =  ' ' + ' '.join(st[end:])
    else: 
        st2 = ''
    p = list(itertools.permutations(part))
    for i in range(0, len(p), skips): #removed tqdm
        t =  st1 + ' '.join(list(p[i])) + st2
        if t in past:
            s = past[t]
        else:
            s =  scorer.get_perplexity(t)
        #if s <= best * tol and t not in past and s<smax:
        if s < best and t not in past:
            print("New Score: ", s, t)
            best = s
            bestt = t
        if t not in past:
            past[t] = s
    return bestt

In [6]:
def generate_neighbor(text, start, end):
    words = text.split(' ')
    part = words[start:end]

    if len(part) < 2:
        return text

    # 隣接状態の生成方法をランダムに選択
    choice = random.choice(['swap', 'reverse'])
    
    if choice == 'swap':
        # ランダムに2つの単語を交換
        i, j = random.sample(range(len(part)), 2)
        part[i], part[j] = part[j], part[i]
    elif choice == 'reverse':
        # ランダムに部分文字列を選んで逆順にする
        if len(part) >= 2:
            part = part[::-1]

    # テキストの前後部分を再構築
    if start > 0:
        st1 = ' '.join(words[:start]) + ' '
    else:
        st1 = ''
    
    if end < len(words):
        st2 = ' ' + ' '.join(words[end:])
    else:
        st2 = ''

    # 新しいテキストを生成
    new_text = st1 + ' '.join(part) + st2
    return new_text

In [7]:
def simulated_annealing_part_perm(st, start=0, end=3, T_initial=100.0, alpha=0.95, T_min=1.0, iterations=100):
    global past
    current_text = st
    current_score = scorer.get_perplexity(current_text)
    best_text = current_text
    best_score = current_score
    T = T_initial
    
    while T > T_min:
        for _ in range(iterations):
            neighbor_text = generate_neighbor(current_text, start, end)
            if neighbor_text in past:
                neighbor_score = past[neighbor_text]
            else:
                neighbor_score = scorer.get_perplexity(neighbor_text)
                past[neighbor_text] = neighbor_score
            
            delta = neighbor_score - current_score
            
            if delta < 0:
                # 改善があれば受け入れる
                current_text = neighbor_text
                current_score = neighbor_score
                if neighbor_score < best_score:
                    best_text = neighbor_text
                    best_score = neighbor_score
                    print(f"New Best Score: {best_score} | Text: {best_text}")
            else:
                # 悪化しても確率的に受け入れる
                probability = math.exp(-delta / T)
                if random.random() < probability:
                    current_text = neighbor_text
                    current_score = neighbor_score
        
        # 温度を冷却
        T *= alpha
        print(f"Temperature: {T:.2f} | Best Score: {best_score}")
    
    return best_text

In [8]:
i = 0
bestt = df['text'][i]
l = len(df['text'][i].split(' '))
for p in range(2, 3):
    for start in range(0,l-p+1):
        print("START: ", start)
        bestt = simulated_annealing_part_perm(bestt, start, start+p, 1)
        df.to_csv("submission.csv", index=False)

df['score'] = df['text'].map(lambda x: scorer.get_perplexity(x))
df.to_csv("submission.csv", index=False)
print(np.mean(df['score']))
df['score']

START:  0
START:  1
START:  2
START:  3
START:  4
START:  5
START:  6
START:  7
START:  8
262.95834068043825


0    468.499121
1    463.726354
2    303.031473
3    209.184454
4     98.260833
5     35.047809
Name: score, dtype: float64